## Assessing the Quality of Gridded Population Data for Quantifying the Population Living in Deprived Communities

Code for the paper "Assessing the Quality of Gridded Population Data for Quantifying the Population Living in Deprived Communities" represented at Machine Learning for the Developing World (ML4D) at NeurIPS 2020. <br>

In [1]:
import fiona
import rasterio
import rasterio.mask
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rasterstats import zonal_stats
import geopandas as gpd
import collections, numpy

In [2]:
#Loading shapefile with deprived areas and converting it to the same CRS as the gridded polutation datasets
shapeoriginal = gpd.read_file("SmallerRegions/GreaterSaoPaulo/GreaterSaoPaulo.shp")
print(shapeoriginal.crs)
shape_epsg  = shapeoriginal.to_crs(epsg=4326)
print(shape_epsg.crs)
print(shape_epsg['CodAGSN'].nunique())
type(shape_epsg)

epsg:4674
epsg:4326
1703


geopandas.geodataframe.GeoDataFrame

In [ ]:
#Create a output path for the data
#outfp = "SmallerRegions/GreaterSaoPaulo/GreaterSaoPaulo_epsg4326.shp"
#shape_epsg.to_file(outfp)
#shape_epsg.plot(figsize=(15,15),edgecolor="blue", facecolor="none")

# WorldPOP

As mentioned in the paper, WorldPOP 2010 gridded population was assessed. The file was accessed on https://www.worldpop.org/ by selecting Individual Contries -> Brazil 2010 -> 100m resolution (the data that was not UN adjusted was used). 

In [3]:
path_popgrid = r"C:/Users/agati/Documents/DatasetsLocal/GriddedPop/WorldPOP/bra_ppp_2010.tif"
popgrid = rasterio.open(path_popgrid)
popgrid.meta

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': -99999.0,
 'width': 54172,
 'height': 46814,
 'count': 1,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(0.0008333333300044304, 0.0, -73.989583022,
        0.0, -0.0008333333300081173, 5.264583514)}

In [4]:
zs_feats = zonal_stats(shape_epsg,'C:/Users/agati/Documents/DatasetsLocal/GriddedPop/WorldPOP/bra_ppp_2010.tif', stats=['sum'], geojson_out=True, raster_out=True)

In [5]:
zs_feats[0]

{'id': '0',
 'type': 'Feature',
 'properties': {'CD_GEOCODM': '3516408',
  'CodAGSN': '35164080001',
  'NM_AGSN': 'VILA BAZU',
  'NM_MUNICIP': 'FRANCO DA ROCHA',
  'uf': '35',
  'sum': 301.90716552734375,
  'mini_raster_array': masked_array(
    data=[[85.9791488647461, 44.90716552734375],
          [84.2720947265625, --],
          [86.74876403808594, --],
          [--, --]],
    mask=[[False, False],
          [False,  True],
          [False,  True],
          [ True,  True]],
    fill_value=1e+20,
    dtype=float32),
  'mini_raster_affine': Affine(0.0008333333300044304, 0.0, -46.73041646422509,
         0.0, -0.0008333333300081173, -23.320416371938443),
  'mini_raster_nodata': -99999.0},
 'geometry': {'type': 'Polygon',
  'coordinates': (((-46.729898999754994, -23.32314699999864),
    (-46.730046000235916, -23.323103999815146),
    (-46.73025199983931, -23.32299200004826),
    (-46.73037400006717, -23.322649000424917),
    (-46.73038699976706, -23.321936999979755),
    (-46.730143

In [6]:
extract = []
for i in range(0,len(zs_feats)):
    extract.append(zs_feats[i]['properties']['CD_GEOCODM'])
    extract.append(zs_feats[i]['properties']['NM_AGSN'])
    extract.append(zs_feats[i]['properties']['CodAGSN'])
    extract.append(zs_feats[i]['properties']['sum'])
array = np.asarray(extract)
array = array.reshape(len(zs_feats),4)
raw_results = pd.DataFrame(array, columns=['CD_GEOCODM','NM_AGSN','CodAGSN','sum'])

In [10]:
#raw_results.to_csv('raw_resultsWorldPOP2010.csv')
#raw_results = pd.read_csv('raw_resultsWorldPOP2010.csv', index_col=0)

In [7]:
print(raw_results.shape)
raw_results.head()

(1703, 4)


,CD_GEOCODM,NM_AGSN,CodAGSN,sum
0,3516408,VILA BAZU,35164080001,301.907
1,3516408,PRETÓRIA,35164080002,3007.59
2,3516408,PARQUE MONTREAL,35164080004,316
3,3516408,VILA PALMARES,35164080005,551.331
4,3516408,ARCO ÍRIS,35164080006,1385


In [8]:
raw_results.dtypes

CD_GEOCODM    object
NM_AGSN       object
CodAGSN       object
sum           object
dtype: object

In [9]:
raw_results['sum'] = raw_results['sum'].astype(float)
raw_results['CodAGSN'] = raw_results['CodAGSN'].astype(float)
print(raw_results.dtypes)

CD_GEOCODM     object
NM_AGSN        object
CodAGSN       float64
sum           float64
dtype: object


In [10]:
raw_results.columns = ['CD_GEOCODM','NM_AGSN','CodAGSN','Estimated_Population']
raw_results.head()

,CD_GEOCODM,NM_AGSN,CodAGSN,Estimated_Population
0,3516408,VILA BAZU,3.516408e+10,301.907166
1,3516408,PRETÓRIA,3.516408e+10,3007.592529
2,3516408,PARQUE MONTREAL,3.516408e+10,316.000000
3,3516408,VILA PALMARES,3.516408e+10,551.330688
4,3516408,ARCO ÍRIS,3.516408e+10,1385.000000


In [11]:
#Reading the groud-truth population values
raw_groundtruth = pd.read_csv("Ground-truth/DataAreasSlumPopulation.csv", index_col=0)
print(raw_groundtruth.columns)
raw_groundtruth.columns = ['CodAGSN', 'DPO_EmAGSN', 'Pop_EmAGSN']
raw_groundtruth.head()

Index(['CD_AGSN', 'DPO_EmAGSN', 'PopEmDPO_EmAGSN'], dtype='object')


,CodAGSN,DPO_EmAGSN,Pop_EmAGSN
1,51034030004,2134,7383
2,51034030005,2709,9498
3,51034030006,1952,6936
4,51034030007,197,715
5,51034030008,1867,6426


In [12]:
results2 =  raw_results.merge(raw_groundtruth,on=['CodAGSN'])

In [13]:
#results2

In [14]:
results2['AbsoluteDifference'] = results2['Estimated_Population']-results2['Pop_EmAGSN']

In [15]:
results2['ProportionalDifference'] = results2['AbsoluteDifference']/results2['Pop_EmAGSN']

In [16]:
#results2.to_csv('resultsWorldPOP2010_repo.csv')

Overall error for all polygons.

In [17]:
results2['Estimated_Population'].sum()

2035865.984515667

In [18]:
results2['Pop_EmAGSN'].sum()

2162368

In [19]:
(results2['Estimated_Population'].sum()-results2['Pop_EmAGSN'].sum())/results2['Pop_EmAGSN'].sum()

-0.05850161280796471

Relative error by range 

In [20]:
a = results2['ProportionalDifference']
conditions = [(a >= -.2) & (a <= 0.2), (a < -0.2) & (a >= -1), (a > 0.2) & (a <= 1), (a < -1), (a > +1)]
choices = ['green', 'yellow', 'orange','purple','red']
res = np.select(conditions, choices)

In [21]:
collections.Counter(res)

Counter({'green': 1135, 'yellow': 268, '0': 140, 'orange': 138, 'red': 22})

# LandScan

As mentioned in the paper, LandScan 2010 gridded population was assessed. The file was accessed on https://landscan.ornl.gov/ on 31/Jul/2020. More details about the dataset can be found on https://landscan.ornl.gov/documentation
    

In [22]:
path_landscan_Reduced = r"clipped.tif"
landscanReduced = rasterio.open(path_landscan_Reduced)
landscanReduced.meta

{'driver': 'GTiff',
 'dtype': 'int32',
 'nodata': 0.0,
 'width': 80,
 'height': 73,
 'count': 1,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(0.008333333333333304, 0.0, -46.95000000000053,
        0.0, -0.008333333333333297, -23.26666666666692)}

In [42]:
#plt.imshow(landscanReduced.read(1), cmap='terrain')

In [43]:
zs_feats = zonal_stats(shape_epsg,'C:/Users/agati/OneDrive - University College Dublin/Experiments/PopGrids/clipped.tif', stats=['sum'], geojson_out=True, raster_out=True)

In [44]:
extract = []
for i in range(0,len(zs_feats)):
#    extract.append(zs_feats[i]['properties']['CD_GEOCODI'])
    extract.append(zs_feats[i]['properties']['CD_GEOCODM'])
    extract.append(zs_feats[i]['properties']['NM_AGSN'])
#    extract.append(zs_feats[i]['properties']['CD_AGSN'])
    extract.append(zs_feats[i]['properties']['CodAGSN'])
    extract.append(zs_feats[i]['properties']['sum'])
array = np.asarray(extract)
array = array.reshape(len(zs_feats),4)
#raw_results = pd.DataFrame(array, columns=['CD_GEOCODI','NM_AGSN','CD_AGSN','sum']) 
raw_results = pd.DataFrame(array, columns=['CD_GEOCODM','NM_AGSN','CodAGSN','sum'])

In [52]:
#raw_results.to_csv('raw_resultsLandScan2010.csv')
#raw_results = pd.read_csv('raw_resultsLandScan2010.csv', index_col=0)

In [45]:
raw_results['sum'] = raw_results['sum'].astype(float)
raw_results['CodAGSN'] = raw_results['CodAGSN'].astype(float)
print(raw_results.dtypes)
raw_results.columns = ['CD_GEOCODM','NM_AGSN','CodAGSN','Estimated_Population']

CD_GEOCODM     object
NM_AGSN        object
CodAGSN       float64
sum           float64
dtype: object


In [47]:
results3 =  raw_results.merge(raw_groundtruth,on=['CodAGSN'])
results3['AbsoluteDifference'] = results3['Estimated_Population']-results3['Pop_EmAGSN']
results3['ProportionalDifference'] = results3['AbsoluteDifference']/results3['Pop_EmAGSN']

In [50]:
results3.to_csv('resultsLandScan2010_repo.csv')

In [48]:
a = results3['ProportionalDifference']
conditions = [(a >= -.2) & (a <= 0.2), (a < -0.2) & (a >= -1), (a > 0.2) & (a <= 1), (a < -1), (a > +1)]
choices = ['green', 'yellow', 'orange','purple','red']
res = np.select(conditions, choices)

In [49]:
collections.Counter(res)

Counter({'red': 50, '0': 1607, 'green': 7, 'orange': 12, 'yellow': 27})

Thank you.